### The Scenario: "Operation Valley Watch"

**Location:** A remote river valley in a NATO Partner nation. Heavy rainfall has triggered flash floods.

#### The Situation
*   **Connectivity:** None. Cell towers are down. The only link to HQ is a low-bandwidth satellite phone (BGAN terminal) that costs $5 per megabyte and is very slow.
*   **The Asset:** A small team of first responders has a drone and a ruggedized laptop (Edge Device).
*   **The Problem:** The drone returned with 2,000 images. The team cannot upload 2,000 images to the cloud to check for damage—it would take days.
*   **The Solution:** An **Edge AI Triage Tool** running locally on the laptop via Gradio.

#### The Workflow
1.  **Ingest:** The drone operator pulls the SD card and plugs it into the laptop.
2.  **Edge Inference:** The Gradio app runs a lightweight, quantized model (ResNet-50 Int8) locally on the laptop's CPU.
3.  **Triage:**
    *   *Class "No Building" / "Safe":* Ignored.
    *   *Class "Flooded":* Flagged.
4.  **Human-in-the-Loop:** The operator reviews *only* the flagged images in the Gradio interface.
5.  **Action:** The operator clicks "Confirm," and *only those specific high-priority images* are compressed and sent via the satellite link to HQ.

In [1]:
import gradio as gr
from transformers import pipeline
from PIL import Image
import torch

In [2]:
device = 0 if torch.cuda.is_available() else -1

In [3]:
classifier = pipeline(
    task="zero-shot-image-classification",
    model="openai/clip-vit-base-patch32",
    device=device
)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cpu


In [4]:
def edge_ai_triage(image):
    """
    Uses OpenAI's CLIP model to classify the image against custom disaster labels.
    Decides whether to transmit data based on the 'Flood' confidence score.
    """
    if image is None:
        return None, "Waiting for drone feed..."

    # Define the specific classes we care about for this mission
    # We can change these text prompts instantly without retraining the model!
    candidate_labels = [
        "flooded residential area",
        "undamaged residential area",
        "forest or empty road",
        "collapsed bridge or infrastructure"
    ]

    # Run Inference
    results = classifier(image, candidate_labels=candidate_labels)

    # Process Results for Gradio (Format: {label: score})
    confidences = {res["label"]: res["score"] for res in results}

    # Get the score specifically for flood or damage to make a decision
    flood_score = confidences.get("flooded residential area", 0)
    damage_score = confidences.get("collapsed bridge or infrastructure", 0)
    total_danger_score = flood_score + damage_score

    # --- DECISION LOGIC (The "Hybrid Strategy") ---
    # This simulates the bandwidth-saving logic

    if total_danger_score > 0.6:
        # High confidence of damage -> Compress and Send immediately
        status = "🔴 CRITICAL ALERT"
        action = f"CONFIDENCE: {total_danger_score:.1%}\nACTION: HIGH PRIORITY UPLOAD via Satellite.\n(Queueing for HQ...)"
    elif total_danger_score > 0.3:
        # Unsure -> Flag for local human review
        status = "🟡 UNCERTAIN"
        action = f"CONFIDENCE: {total_danger_score:.1%}\nACTION: FLAGGED FOR HUMAN REVIEW.\n(Do not transmit yet)"
    else:
        # Safe/Empty -> Drop to save bandwidth
        status = "🟢 LOW PRIORITY"
        action = f"CONFIDENCE: {total_danger_score:.1%}\nACTION: DISCARD / LOG LOCALLY.\n(Bandwidth conserved)"

    return confidences, f"{status}\n\n{action}"

In [5]:
custom_css = """
body {background-color: #1e1e1e; color: #e0e0e0;}
.gradio-container {font-family: 'Courier New', monospace;}
h1 {color: #ff4b4b; text-align: center;}
.stat-box {border: 2px solid #555; padding: 10px; border-radius: 5px;}
"""

In [6]:
with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:

    gr.Markdown(
        """
        # 🚁 NATO SPS: EDGE TRIAGE TERMINAL
        ### Operation Valley Watch | Model: CLIP-ViT-Base | Link Status: 📶 LOW (BGAN)
        """
    )

    with gr.Row():
        with gr.Column(scale=1):
            input_img = gr.Image(label="Input Drone Feed", type="pil", height=300)
            analyze_btn = gr.Button("RUN DIAGNOSTIC", variant="stop", size="lg")

        with gr.Column(scale=1):
            # Output for the probability distribution
            lbl_output = gr.Label(num_top_classes=4, label="AI Scene Analysis")

            # Output for the Tactical Decision
            decision_output = gr.Textbox(
                label="Tactical Recommendation (Bandwidth Manager)",
                lines=5,
                elem_classes="stat-box"
            )

    # Link the logic
    analyze_btn.click(
        fn=edge_ai_triage,
        inputs=input_img,
        outputs=[lbl_output, decision_output]
    )

    gr.Markdown(
        """
        ---
        **System Logic:**
        1. **AI Inference:** Runs locally on rugged laptop (Simulated via Hugging Face).
        2. **Filter:** Only transmits images with >60% damage probability.
        3. **Result:** Reduces satellite data usage by ~90%.
        """
    )

In [7]:
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a498f91dec89ca0ffc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
